In [21]:
import sys
print (sys.version)

2.7.12 (default, Nov 19 2016, 06:48:10) 
[GCC 5.4.0 20160609]


In [23]:
import numpy as np
import pandas as pd
properties_file = '../input/properties_2016.csv'
training_file = '../input/train_2016_v2.csv'
properties = pd.read_csv(properties_file, dtype={
    'fireplaceflag': np.bool, 'hashottuborspa': np.bool,
    'propertycountylandusecode': np.str,
    'propertyzoningdesc': np.str}, converters={
    'taxdelinquencyflag': lambda x: np.bool(True) if x == 'Y' else np.bool(False)})  # avoid mixed type warning
print ('loaded properties from ' + properties_file)
print ('properties shape: ' + str(properties.shape))
train = pd.read_csv(training_file)
print ('loaded transaction data from ' + training_file)
print ('train shape: ' + str(train.shape))

loaded properties from ../input/properties_2016.csv
properties shape: (2985217, 58)
loaded transaction data from ../input/train_2016_v2.csv
train shape: (90275, 3)


loaded properties from ../input/properties_2016.csv
properties shape: (2985217, 58)
loaded transaction data from ../input/train_2016_v2.csv
train shape: (90275, 3)


In [24]:
# take the log of select columns to reduce skew

log_columns = ['landtaxvaluedollarcnt', 'structuretaxvaluedollarcnt', 'taxamount', 'taxvaluedollarcnt',
               'calculatedfinishedsquarefeet']
properties[log_columns].describe()

,landtaxvaluedollarcnt,structuretaxvaluedollarcnt,taxamount,taxvaluedollarcnt,calculatedfinishedsquarefeet
count,2.917484e+06,2.930235e+06,2.953967e+06,2.942667e+06,2.929652e+06
mean,2.524780e+05,1.708836e+05,5.377607e+03,4.204790e+05,1.827162e+03
std,4.450132e+05,4.020683e+05,9.183107e+03,7.263467e+05,1.819780e+03
min,1.000000e+00,1.000000e+00,1.340000e+00,1.000000e+00,1.000000e+00
25%,7.483600e+04,7.480000e+04,2.461070e+03,1.796750e+05,1.213000e+03
50%,1.670420e+05,1.225900e+05,3.991780e+03,3.060860e+05,1.572000e+03
75%,3.069180e+05,1.968890e+05,6.201005e+03,4.880000e+05,2.136000e+03
max,9.024622e+07,2.514860e+08,3.458861e+06,2.827860e+08,9.525760e+05


In [25]:
for column_name in log_columns:
    properties[column_name] = properties[column_name].apply(lambda x: np.log(x) if pd.notnull(x) else x)
properties[log_columns].describe()

,landtaxvaluedollarcnt,structuretaxvaluedollarcnt,taxamount,taxvaluedollarcnt,calculatedfinishedsquarefeet
count,2.917484e+06,2.930235e+06,2.953967e+06,2.942667e+06,2.929652e+06
mean,1.188352e+01,1.168654e+01,8.234322e+00,1.255060e+01,7.396858e+00
std,1.119799e+00,8.475942e-01,8.526816e-01,9.425025e-01,4.484323e-01
min,0.000000e+00,0.000000e+00,2.926696e-01,0.000000e+00,0.000000e+00
25%,1.122305e+01,1.122257e+01,7.808351e+00,1.209890e+01,7.100852e+00
50%,1.202600e+01,1.171660e+01,8.291993e+00,1.263162e+01,7.360104e+00
75%,1.263434e+01,1.219040e+01,8.732467e+00,1.309807e+01,7.666690e+00
max,1.831805e+01,1.934290e+01,1.505645e+01,1.946020e+01,1.376693e+01


In [26]:
# these are either one value or null, so we can tell the model that they're Boolean

true_false_columns = ['hashottuborspa','fireplaceflag']
properties[true_false_columns].describe()

,hashottuborspa,fireplaceflag
count,69014,5163
unique,1,1
top,True,True
freq,69014,5163


In [27]:
for column_name in true_false_columns:
    properties[column_name] = properties[column_name].apply(lambda x: False if pd.isnull(x) else True)
properties[true_false_columns].describe()

,hashottuborspa,fireplaceflag
count,2985217,2985217
unique,2,2
top,False,False
freq,2916203,2980054


In [28]:
# transform tax delinquency year
print('unique tax delinquency year values: ' + str(properties['taxdelinquencyyear'].unique()))

unique tax delinquency year values: [ nan  13.  15.  11.  14.   9.  10.   8.  12.   7.   6.   2.  92.   5.   3.
   4.  95.  91.  93.  98.  82.  79.  94.  97.  90.  99.   0.   1.  87.  84.
  86.  88.  96.]


In [29]:
properties['taxdelinquencyyear'] = properties['taxdelinquencyyear'].apply(
    lambda x: (17 - x if x < 20 else 117 - x) if pd.notnull(x) else x)
# print('unique tax delinquency year values: ' + str(properties['taxdelinquencyyear'].unique()))

In [30]:
print('unique tax delinquency year values: ' + str(properties['taxdelinquencyyear'].unique()))

unique tax delinquency year values: [ nan   4.   2.   6.   3.   8.   7.   9.   5.  10.  11.  15.  25.  12.  14.
  13.  22.  26.  24.  19.  35.  38.  23.  20.  27.  18.  17.  16.  30.  33.
  31.  29.  21.]


In [31]:
# scale the latitude and longitude using the min-max scaler
location_columns = ['latitude', 'longitude']
properties[location_columns].describe()

,latitude,longitude
count,2.973780e+06,2.973780e+06
mean,3.400147e+07,-1.182019e+08
std,2.433812e+05,3.453171e+05
min,3.332439e+07,-1.194758e+08
25%,3.382768e+07,-1.183930e+08
50%,3.400825e+07,-1.181725e+08
75%,3.416186e+07,-1.179495e+08
max,3.481965e+07,-1.175543e+08


In [32]:
from sklearn.preprocessing import MinMaxScaler

min_max_scaler = MinMaxScaler(copy=True)
scaled_columns = list()
for column_name in location_columns:
    mean_value = properties[column_name].mean()
    properties[column_name].fillna(inplace=True, value=mean_value)
    scaled_columns.append(column_name)
properties[scaled_columns] = min_max_scaler.fit_transform(properties[scaled_columns])
properties[location_columns].describe()

,latitude,longitude
count,2.985217e+06,2.985217e+06
mean,4.528174e-01,6.629559e-01
std,1.624562e-01,1.793710e-01
min,0.000000e+00,0.000000e+00
25%,3.369858e-01,5.640387e-01
50%,4.564712e-01,6.773080e-01
75%,5.597367e-01,7.939581e-01
max,1.000000e+00,1.000000e+00


In [33]:
# label encoding for several coded features
from sklearn.preprocessing import LabelEncoder
label_encode_columns = ['propertycountylandusecode', 'propertyzoningdesc', 'fips', 'regionidzip']
for column_name in label_encode_columns:
    if column_name in ['fips', 'regionidzip']:
        properties[column_name] = properties[column_name].fillna('ZZZ')
    label_encoder = LabelEncoder()
    label_encoder.fit(list(properties[column_name].values))
    properties[column_name] = label_encoder.transform(list(properties[column_name].values))

In [34]:
properties.describe()

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,yardbuildingsqft26,yearbuilt,numberofstories,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyyear,censustractandblock
count,2.985217e+06,811519.000000,6061.000000,1628.000000,2.973755e+06,2.973767e+06,12629.000000,1.938488e+06,2.856305e+06,17096.0,...,2647.000000,2.925289e+06,682069.000000,2.930235e+06,2.942667e+06,2.973778e+06,2.917484e+06,2.953967e+06,56464.000000,2.910091e+06
mean,1.332586e+07,1.931166,7.202607,646.883292,2.209143e+00,3.088949e+00,3.725948,5.784787e+00,2.299263e+00,66.0,...,278.296562,1.964262e+03,1.401464,1.168654e+01,1.255060e+01,2.014999e+03,1.188352e+01,8.234322e+00,3.171348,6.048431e+13
std,7.909966e+06,3.148587,2.436290,538.793473,1.077754e+00,1.275859e+00,0.501700,1.805352e+00,1.000736e+00,0.0,...,369.731508,2.344132e+01,0.539076,8.475942e-01,9.425025e-01,3.683161e-02,1.119799e+00,8.526816e-01,1.735831,3.249035e+11
min,1.071172e+07,1.000000,2.000000,20.000000,0.000000e+00,0.000000e+00,1.000000,1.000000e+00,1.000000e+00,66.0,...,10.000000,1.801000e+03,1.000000,0.000000e+00,0.000000e+00,2.000000e+03,0.000000e+00,2.926696e-01,2.000000,-1.000000e+00
25%,1.164371e+07,1.000000,7.000000,272.000000,2.000000e+00,2.000000e+00,3.000000,4.000000e+00,2.000000e+00,66.0,...,96.000000,1.950000e+03,1.000000,1.122257e+01,1.209890e+01,2.015000e+03,1.122305e+01,7.808351e+00,2.000000,6.037400e+13
50%,1.254509e+07,1.000000,7.000000,534.000000,2.000000e+00,3.000000e+00,4.000000,7.000000e+00,2.000000e+00,66.0,...,168.000000,1.963000e+03,1.000000,1.171660e+01,1.263162e+01,2.015000e+03,1.202600e+01,8.291993e+00,3.000000,6.037572e+13
75%,1.409712e+07,1.000000,7.000000,847.250000,3.000000e+00,4.000000e+00,4.000000,7.000000e+00,3.000000e+00,66.0,...,320.000000,1.981000e+03,2.000000,1.219040e+01,1.309807e+01,2.015000e+03,1.263434e+01,8.732467e+00,3.000000,6.059042e+13
max,1.696019e+08,13.000000,27.000000,8516.000000,2.000000e+01,2.000000e+01,5.000000,1.200000e+01,2.000000e+01,66.0,...,6141.000000,2.015000e+03,41.000000,1.934290e+01,1.946020e+01,2.016000e+03,1.831805e+01,1.505645e+01,38.000000,4.830301e+14
